In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemma2 Fine Tuning with LoRA and deployment on Vertex AI  with vLLM

Based on [model_garden_gemma2_finetuning_on_vertex.ipynb](https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_gemma2_finetuning_on_vertex.ipynb)

![gemma2 fine tuning architecture](images/1.3-gemma2-fine-tuning.png)

## Overview

This notebook demonstrates finetuning and deploying Gemma 2 models with [Vertex AI Custom Training Job](https://cloud.google.com/vertex-ai/docs/training/create-custom-job). All of the examples in this notebook use parameter efficient finetuning methods [PEFT (LoRA)](https://github.com/huggingface/peft) to reduce training and storage costs. LoRA (Low-Rank Adaptation) is one approach of Parameter Efficient FineTuning (PEFT), where pretrained model weights are frozen and rank decomposition matrices representing the change in model weights are trained during finetuning. Read more about LoRA in the following publication: [Hu, E.J., Shen, Y., Wallis, P., Allen-Zhu, Z., Li, Y., Wang, S., Wang, L. and Chen, W., 2021. Lora: Low-rank adaptation of large language models. *arXiv preprint arXiv:2106.09685*](https://arxiv.org/abs/2106.09685).


After tuning, we can deploy models on Vertex with GPU.


### Objective

- Finetune and deploy Gemma 2 models with Vertex AI Custom Training Jobs.
- Send prediction requests to your finetuned Gemma 2 model.

### File a bug

File a bug on [GitHub](https://github.com/GoogleCloudPlatform/vertex-ai-samples/issues/new) if you encounter any issue with the notebook.

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing), [Cloud Storage pricing](https://cloud.google.com/storage/pricing), and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

## Before you begin

### Install Python Packages for Finetuning

Install google-cloud-aiplatform and dataset validation packages and restart the session.

In [ ]:
! pip install --upgrade --quiet 'google-cloud-aiplatform>=1.66.0' accelerate==0.31.0 transformers==4.43.1 datasets==2.19.2

Load local tensorboard

In [ ]:
%load_ext tensorboard

### Setup Google Cloud Project

1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

2. For finetuning, **[click here](https://console.cloud.google.com/iam-admin/quotas?location=us-central1&metric=aiplatform.googleapis.com%2Frestricted_image_training_nvidia_a100_80gb_gpus)** to check if your project already has the required 8 Nvidia A100 80 GB GPUs in the us-central1 region. If yes, then run this notebook in the us-central1 region. If you do not have 8 Nvidia A100 80 GPUs or have more GPU requirements than this, then schedule your job with Nvidia H100 GPUs via Dynamic Workload Scheduler using [these instructions](https://cloud.google.com/vertex-ai/docs/training/schedule-jobs-dws). For Dynamic Workload Scheduler, check the [us-central1](https://console.cloud.google.com/iam-admin/quotas?location=us-central1&metric=aiplatform.googleapis.com%2Fcustom_model_training_preemptible_nvidia_h100_gpus) or [europe-west4](https://console.cloud.google.com/iam-admin/quotas?location=europe-west4&metric=aiplatform.googleapis.com%2Fcustom_model_training_preemptible_nvidia_h100_gpus) quota for Nvidia H100 GPUs. If you do not have enough GPUs, then you can follow [these instructions](https://cloud.google.com/docs/quotas/view-manage#viewing_your_quota_console) to request quota.

3. For serving, **[click here](https://console.cloud.google.com/iam-admin/quotas?location=us-central1&metric=aiplatform.googleapis.com%2Fcustom_model_serving_nvidia_l4_gpus)** to check if your project already has the required 1 L4 GPU in the us-central1 region.  If yes, then run this notebook in the us-central1 region. If you need more L4 GPUs for your project, then you can follow [these instructions](https://cloud.google.com/docs/quotas/view-manage#viewing_your_quota_console) to request more. Alternatively, if you want to run predictions with A100 80GB or H100 GPUs, we recommend using the regions listed below. **NOTE:** Make sure you have associated quota in selected regions. Click the links to see your current quota for each GPU type: [Nvidia A100 80GB](https://console.cloud.google.com/iam-admin/quotas?metric=aiplatform.googleapis.com%2Fcustom_model_serving_nvidia_a100_80gb_gpus), [Nvidia H100 80GB](https://console.cloud.google.com/iam-admin/quotas?metric=aiplatform.googleapis.com%2Fcustom_model_serving_nvidia_h100_gpus).

> | Machine Type | Accelerator Type | Recommended Regions |
| ----------- | ----------- | ----------- | 
| a2-ultragpu-1g | 1 NVIDIA_A100_80GB | us-central1, us-east4, europe-west4, asia-southeast1, us-east4 |
| a3-highgpu-2g | 2 NVIDIA_H100_80GB | us-west1, asia-southeast1 |
| a3-highgpu-4g | 4 NVIDIA_H100_80GB | us-west1, asia-southeast1 |
| a3-highgpu-8g | 8 NVIDIA_H100_80GB | us-central1, us-west1, europe-west4, asia-southeast1 |

4. **[Optional]** [Create a Cloud Storage bucket](https://cloud.google.com/storage/docs/creating-buckets) for storing experiment outputs. Set the BUCKET_URI for the experiment environment. The specified Cloud Storage bucket (`BUCKET_URI`) should be located in the same region as where the notebook was launched. Note that a multi-region bucket (eg. "us") is not considered a match for a single region covered by the multi-region range (eg. "us-central1"). If not set, a unique GCS bucket will be created instead.

In [ ]:
BUCKET_URI = "gs://"  # @param {type:"string"}

5. **[Optional]** Set region. If not set, the region will be set automatically according to Colab Enterprise environment.

In [ ]:
REGION = ""  # @param {type:"string"}

Download samples

# ! rm -rf vertex-ai-samples
! git clone https://github.com/GoogleCloudPlatform/vertex-ai-samples.git

Select the right commit for this example

In [ ]:
! cd vertex-ai-samples && git reset --hard 0727e19520cf7957bceb701c248221bd3dbe4f1f

Import the modules

In [ ]:
import importlib
import os
import uuid
from datetime import datetime
from typing import Tuple

from google.cloud import aiplatform
from google.cloud.aiplatform.compat.types import \
    custom_job as gca_custom_job_compat

common_util = importlib.import_module(
    "vertex-ai-samples.community-content.vertex_model_garden.model_oss.notebook_util.common_util"
)

Define constants

In [ ]:
# Get the default cloud project id.
PROJECT_ID = os.environ["GOOGLE_CLOUD_PROJECT"]

# Get the default region for launching jobs.
if not REGION:
    REGION = os.environ["GOOGLE_CLOUD_REGION"]

# Enable the Vertex AI API and Compute Engine API, if not already.
print("Enabling Vertex AI API and Compute Engine API.")
! gcloud services enable aiplatform.googleapis.com compute.googleapis.com

# Cloud Storage bucket for storing the experiment artifacts.
# A unique GCS bucket will be created for the purpose of this notebook. If you
# prefer using your own GCS bucket, change the value yourself below.
now = datetime.now().strftime("%Y%m%d%H%M%S")
BUCKET_NAME = "/".join(BUCKET_URI.split("/")[:3])

if BUCKET_URI is None or BUCKET_URI.strip() == "" or BUCKET_URI == "gs://":
    BUCKET_URI = f"gs://{PROJECT_ID}-tmp-{now}-{str(uuid.uuid4())[:4]}"
    BUCKET_NAME = "/".join(BUCKET_URI.split("/")[:3])
    ! gsutil mb -l {REGION} {BUCKET_URI}
else:
    assert BUCKET_URI.startswith("gs://"), "BUCKET_URI must start with `gs://`."
    shell_output = ! gsutil ls -Lb {BUCKET_NAME} | grep "Location constraint:" | sed "s/Location constraint://"
    bucket_region = shell_output[0].strip().lower()
    if bucket_region != REGION:
        raise ValueError(
            "Bucket region %s is different from notebook region %s"
            % (bucket_region, REGION)
        )
print(f"Using this GCS Bucket: {BUCKET_URI}")

STAGING_BUCKET = os.path.join(BUCKET_URI, "temporal")
MODEL_BUCKET = os.path.join(BUCKET_URI, "gemma2")

Gets the default service account

In [ ]:
shell_output = ! gcloud projects describe $PROJECT_ID
project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"
print("Using this default Service Account:", SERVICE_ACCOUNT)

Provision permissions to the SERVICE_ACCOUNT with the GCS bucket

In [ ]:
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.admin $BUCKET_NAME

! gcloud config set project $PROJECT_ID
! gcloud projects add-iam-policy-binding --no-user-output-enabled {PROJECT_ID} --member=serviceAccount:{SERVICE_ACCOUNT} --role="roles/storage.admin"
! gcloud projects add-iam-policy-binding --no-user-output-enabled {PROJECT_ID} --member=serviceAccount:{SERVICE_ACCOUNT} --role="roles/aiplatform.user"

### Access Gemma2 models

You must provide a Hugging Face User Access Token (read) to access the Gemma 2 models. You can follow the [Hugging Face documentation](https://huggingface.co/docs/hub/en/security-tokens) to create a **read** access token and put it in the `HF_TOKEN` field below.

In [ ]:
HF_TOKEN = ""  # @param {type:"string", isTemplate:true}
assert HF_TOKEN, "Provide a read HF_TOKEN to load models from Hugging Face."

model_path_prefix = "google/"

In [ ]:
models, endpoints = {}, {}

## Finetune with HuggingFace PEFT and Deploy with vLLM on GPUs

### Set dataset

Use the Vertex AI SDK to create and run the custom training jobs.

This notebook uses [timdettmers/openassistant-guanaco](https://huggingface.co/datasets/timdettmers/openassistant-guanaco) dataset as an example.
You can set `dataset_name` to any existing [Hugging Face dataset](https://huggingface.co/datasets) name, and set `instruct_column_in_dataset` to the name of the dataset column containing training data. The [timdettmers/openassistant-guanaco](https://huggingface.co/datasets/timdettmers/openassistant-guanaco) has only one column `text`, and therefore we set `instruct_column_in_dataset` to `text` in this notebook.

### (Optional) Prepare a custom JSONL dataset for finetuning

You can prepare a JSONL file where each line is a valid JSON string as your custom training dataset. For example, here is one line from the [timdettmers/openassistant-guanaco](https://huggingface.co/datasets/timdettmers/openassistant-guanaco) dataset:
```
{"text": "### Human: Hola### Assistant: \u00a1Hola! \u00bfEn qu\u00e9 puedo ayudarte hoy?"}
```

The JSON object has a key `text`, which should match `instruct_column_in_dataset`; The value should be one training data point, i.e. a string. After you prepared your JSONL file, you can either upload it to [Hugging Face datasets](https://huggingface.co/datasets) or [Google Cloud Storage](https://cloud.google.com/storage).

- To upload a JSONL dataset to [Hugging Face datasets](https://huggingface.co/datasets), follow the instructions on [Uploading Datasets](https://huggingface.co/docs/hub/en/datasets-adding). Then, set `dataset_name` to the name of your newly created dataset on Hugging Face.

- To upload a JSONL dataset to [Google Cloud Storage](https://cloud.google.com/storage), follow the instructions on [Upload objects from a filesystem](https://cloud.google.com/storage/docs/uploading-objects). Then, set `dataset_name` to the `gs://` URI to your JSONL file. For example: `gs://cloud-samples-data/vertex-ai/model-evaluation/peft_train_sample.jsonl`.

Optionally update the `instruct_column_in_dataset` field below if your JSON objects use a key other than the default `text`.

### (Optional) Format your data with custom JSON template

Sometimes, your dataset might have multiple text columns and you want to construct the training data with a template. You can prepare a JSON template in the following format:

```
{
  "description": "Template that accepts text-bison format.",
  "source": "https://cloud.google.com/vertex-ai/generative-ai/docs/models/tune-text-models-supervised#dataset-format",
  "prompt_input": "\n\n<|start_header_id|>user<|end_header_id|>\n\n{input_text}<|eot_id|>\n\n<|start_header_id|>assistant<|end_header_id|>\n\n{output_text}<|eot_id|>",
  "instruction_separator": "<|start_header_id|>user<|end_header_id|>\n\n",
  "response_separator": "<|start_header_id|>assistant<|end_header_id|>\n\n"
}
```

As an example, the template above can be used to format the following training data (this line comes from `gs://cloud-samples-data/vertex-ai/model-evaluation/peft_train_sample.jsonl`):

```
{"input_text":"TRANSCRIPT: \nREASON FOR EVALUATION:,\n\n LABEL:","output_text":"Chiropractic"}
```

This example template simply concatenates `input_text` with `output_text` with some special tokens in between.
To try such custom dataset, you can make the following changes:
1. Set `template` to `llama3-text-bison`
1. Set `train_dataset_name` to `gs://cloud-samples-data/vertex-ai/model-evaluation/peft_train_sample.jsonl`
1. Set `train_split_name` to `train`
1. Set `eval_dataset_name` to `gs://cloud-samples-data/vertex-ai/model-evaluation/peft_eval_sample.jsonl`
1. Set `eval_split_name` to `train` (**NOT** `test`)
1. Set `instruct_column_in_dataset` as `input_text`.

In [ ]:
# Template name or gs:// URI to a custom template.
template = "openassistant-guanaco"  # @param {type:"string"}

# Hugging Face dataset name or gs:// URI to a custom JSONL dataset.
train_dataset_name = "timdettmers/openassistant-guanaco"  # @param {type:"string"}
train_split_name = "train"  # @param {type:"string"}
eval_dataset_name = "timdettmers/openassistant-guanaco"  # @param {type:"string"}
eval_split_name = "test"  # @param {type:"string"}

# Name of the dataset column containing training text input.
instruct_column_in_dataset = "text"  # @param {type:"string"}

### Set model

Select a model variant of Gemma 2.

In [ ]:
base_model_id = "gemma-2-2b-it"  # @param ["gemma-2-2b", "gemma-2-2b-it", "gemma-2-9b", "gemma-2-9b-it", "gemma-2-27b", "gemma-2-27b-it"] {isTemplate: true}
pretrained_model_id = os.path.join(model_path_prefix, base_model_id)

### Validate Dataset with Template

This section validates the train and eval datasets with the template before starting the fine tuning process.

In [ ]:
import transformers

dataset_validation_util = importlib.import_module(
    "vertex-ai-samples.community-content.vertex_model_garden.model_oss.notebook_util.dataset_validation_util"
)

if dataset_validation_util.is_gcs_path(pretrained_model_id):
    # Download tokenizer.
    ! mkdir tokenizer
    ! gsutil cp {pretrained_model_id}/tokenizer.json ./tokenizer
    ! gsutil cp {pretrained_model_id}/config.json ./tokenizer
    tokenizer_path = "./tokenizer"
    access_token = ""
else:
    tokenizer_path = pretrained_model_id
    access_token = HF_TOKEN

tokenizer = transformers.AutoTokenizer.from_pretrained(
    tokenizer_path,
    trust_remote_code=False,
    use_fast=True,
    token=access_token,
)

# Validate the train dataset.
dataset_validation_util.validate_dataset_with_template(
    dataset_name=train_dataset_name,
    split=train_split_name,
    input_column=instruct_column_in_dataset,
    template=template,
    use_multiprocessing=False,
    tokenizer=tokenizer,
)

# Validate the eval dataset.
dataset_validation_util.validate_dataset_with_template(
    dataset_name=eval_dataset_name,
    split=eval_split_name,
    input_column=instruct_column_in_dataset,
    template=template,
    use_multiprocessing=False,
    tokenizer=tokenizer,

### Finetune

This section demonstrates how to finetune the Gemma 2 model and merge the finetuned LoRA adapter with the base model on Vertex AI. It uses the Vertex AI SDK to create and run the custom training jobs.

The training job takes approximately between 10 to 20 mins to set-up. Once done, the training job is expected to take around 20 mins with the default configuration. To find the training time, throughput, and memory usage of your training job, you can go to the training logs and check the log line of the last training epoch.

**Note**:
1. We recommend setting `finetuning_precision_mode` to `4bit` because it enables using fewer hardware resources for finetuning.
1. If `max_steps > 0`, it takes precedence over `epochs`. One can set a small `max_steps` value to quickly check the pipeline.

Accelerator type to use for training.

In [ ]:
accelerator_type = "NVIDIA_A100_80GB"  # @param ["NVIDIA_A100_80GB", "NVIDIA_H100_80GB"]

In [ ]:
# The pre-built training docker image.
if accelerator_type == "NVIDIA_A100_80GB":
    repo = "us-docker.pkg.dev/vertex-ai-restricted"
    is_restricted_image = True
    is_dynamic_workload_scheduler = False
    dws_kwargs = {}
else:
    repo = "us-docker.pkg.dev/vertex-ai"
    is_restricted_image = False
    is_dynamic_workload_scheduler = True
    dws_kwargs = {
        "max_wait_duration": 1800,  # 30 minutes
        "scheduling_strategy": gca_custom_job_compat.Scheduling.Strategy.FLEX_START,
    }

TRAIN_DOCKER_URI = (
    f"{repo}/vertex-vision-model-garden-dockers/pytorch-peft-train:stable_20240909"
)

# Worker pool spec.
if accelerator_type == "NVIDIA_A100_80GB":
    per_node_accelerator_count = 8
    machine_type = "a2-ultragpu-8g"
elif accelerator_type == "NVIDIA_H100_80GB":
    per_node_accelerator_count = 8
    machine_type = "a3-highgpu-8g"
else:
    raise ValueError(
        f"Recommended machine settings not found for: {accelerator_type}. To use another accelerator type, edit this code block to pass in an appropriate `machine_type`, `accelerator_type`, and `per_node_accelerator_count` to the deploy_model_vllm function by clicking `Show Code` and then modifying the code."
    )

Parameters

In [ ]:
# Batch size for finetuning.
per_device_train_batch_size = 1  # @param{type:"integer"}
# Number of updates steps to accumulate the gradients for, before performing a backward/update pass.
gradient_accumulation_steps = 4  # @param{type:"integer"}
# Maximum sequence length.
max_seq_length = 4096  # @param{type:"integer"}
# Setting a positive `max_steps` here will override `num_epochs`.
max_steps = -1  # @param{type:"integer"}
num_epochs = 1.0  # @param{type:"number"}
# Precision mode for finetuning.
finetuning_precision_mode = "4bit"  # @param ["4bit", "8bit", "float16"]
# Learning rate.
learning_rate = 5e-5  # @param{type:"number"}
# The scheduler type to use.
lr_scheduler_type = "cosine"  # @param{type:"string"}
# LoRA parameters.
lora_rank = 16  # @param{type:"integer"}
lora_alpha = 32  # @param{type:"integer"}
lora_dropout = 0.05  # @param{type:"number"}
# Activates gradient checkpointing for the current model (may be referred to as activation checkpointing or checkpoint activations in other frameworks).
enable_gradient_checkpointing = True
# Attention implementation to use in the model.
attn_implementation = "eager"
# The optimizer for which to schedule the learning rate.
optimizer = "paged_adamw_32bit"
# Define the proportion of training to be dedicated to a linear warmup where learning rate gradually increases.
warmup_ratio = "0.01"
# The list or string of integrations to report the results and logs to.
report_to = "tensorboard"
# Number of updates steps before two checkpoint saves.
save_steps = 10
# Number of update steps between two logs.
logging_steps = save_steps
# Train precision of the model.
train_precision = "bfloat16"

In [ ]:
replica_count = 1

common_util.check_quota(
    project_id=PROJECT_ID,
    region=REGION,
    accelerator_type=accelerator_type,
    accelerator_count=per_node_accelerator_count * replica_count,
    is_for_training=True,
    is_restricted_image=is_restricted_image,
    is_dynamic_workload_scheduler=is_dynamic_workload_scheduler,
)

In [ ]:
base_output_dir = os.path.join(STAGING_BUCKET, job_name)
# Create a GCS folder to store the LORA adapter.
lora_output_dir = os.path.join(base_output_dir, "adapter")
# Create a GCS folder to store the merged model with the base model and the
# finetuned LORA adapter.
merged_model_output_dir = os.path.join(base_output_dir, "merged-model")

eval_args = [
    f"--eval_dataset_path={eval_dataset_name}",
    f"--eval_column={instruct_column_in_dataset}",
    f"--eval_template={template}",
    f"--eval_split={eval_split_name}",
    f"--eval_steps={save_steps}",
    "--eval_tasks=builtin_eval",
    "--eval_metric_name=loss",
]

train_job_args = [
    "--config_file=vertex_vision_model_garden_peft/deepspeed_zero2_8gpu.yaml",
    "--task=instruct-lora",
    "--completion_only=True",
    f"--pretrained_model_id={pretrained_model_id}",
    f"--dataset_name={train_dataset_name}",
    f"--train_split_name={train_split_name}",
    f"--instruct_column_in_dataset={instruct_column_in_dataset}",
    f"--output_dir={lora_output_dir}",
    f"--merge_base_and_lora_output_dir={merged_model_output_dir}",
    f"--per_device_train_batch_size={per_device_train_batch_size}",
    f"--gradient_accumulation_steps={gradient_accumulation_steps}",
    f"--lora_rank={lora_rank}",
    f"--lora_alpha={lora_alpha}",
    f"--lora_dropout={lora_dropout}",
    f"--max_steps={max_steps}",
    f"--max_seq_length={max_seq_length}",
    f"--learning_rate={learning_rate}",
    f"--lr_scheduler_type={lr_scheduler_type}",
    f"--precision_mode={finetuning_precision_mode}",
    f"--train_precision={train_precision}",
    f"--enable_gradient_checkpointing={enable_gradient_checkpointing}",
    f"--num_epochs={num_epochs}",
    f"--attn_implementation={attn_implementation}",
    f"--optimizer={optimizer}",
    f"--warmup_ratio={warmup_ratio}",
    f"--report_to={report_to}",
    f"--logging_output_dir={base_output_dir}",
    f"--save_steps={save_steps}",
    f"--logging_steps={logging_steps}",
    f"--template={template}",
    f"--huggingface_access_token={HF_TOKEN}",
] + eval_args

In [ ]:
# Pass training arguments and launch job.
train_job = aiplatform.CustomContainerTrainingJob(
    display_name=job_name,
    container_uri=TRAIN_DOCKER_URI,
)

print("Running training job with args:")
print(" \\\n".join(train_job_args))
train_job.run(
    args=train_job_args,
    replica_count=replica_count,
    machine_type=machine_type,
    accelerator_type=accelerator_type,
    accelerator_count=per_node_accelerator_count,
    boot_disk_size_gb=500,
    service_account=SERVICE_ACCOUNT,
    base_output_dir=base_output_dir,
    sync=False,  # Non-blocking call to run.
    **dws_kwargs,
)

# Wait until resource has been created.
train_job.wait_for_resource_creation()

print("LoRA adapter will be saved in:", lora_output_dir)
print("Trained and merged models will be saved in:", merged_model_output_dir)

### Run Tensorboard

This section shows how to launch TensorBoard in a [Cloud Shell](https://cloud.google.com/shell/docs).
1. Click the Cloud Shell icon(![terminal](https://github.com/google/material-design-icons/blob/master/png/action/terminal/materialicons/24dp/1x/baseline_terminal_black_24dp.png?raw=true)) on the top right to open the Cloud Shell.
2. Copy the `tensorboard` command shown below by running this cell.
3. Paste and run the command in the Cloud Shell to launch TensorBoard.
4. Once the command runs (You may have to click `Authorize` if prompted), click the link starting with `http://localhost`.

Note: You may need to wait around 10 minutes after the job starts in order for the TensorBoard logs to be written to the GCS bucket.

In [ ]:
print(f"Command to copy: tensorboard --logdir {base_output_dir}/logs")

### Deploy

This section uploads the model to Model Registry and deploys it on the Endpoint. It takes 15 minutes to 1 hour to finish.

In [ ]:
if train_job.end_time is None:
    print("Waiting for the training job to finish...")
    train_job.wait()
    print("The training job has finished.")

print("Deploying models in:", merged_model_output_dir)

The pre-built serving docker image for vLLM.

In [ ]:
VLLM_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-vllm-serve:20240815_1634_RC00"

Accelerator type to use for serving. Find Vertex AI prediction supported accelerators and regions [here](https://cloud.google.com/vertex-ai/docs/predictions/configure-compute).

In [ ]:
accelerator_type = "NVIDIA_L4"  # @param ["NVIDIA_L4"] {isTemplate: true}

Machine type

In [ ]:
if "2b" in base_model_id:
    if accelerator_type == "NVIDIA_L4":
        # Sets 1 L4 (24G) to deploy Gemma 2 2B models.
        machine_type = "g2-standard-12"
        accelerator_count = 1
    else:
        raise ValueError(
            "Recommended machine settings not found for accelerator type: %s"
            % accelerator_type
        )
elif "9b" in base_model_id:
    if accelerator_type == "NVIDIA_L4":
        # Sets 2 L4 (24G) to deploy Gemma 2 9B models.
        machine_type = "g2-standard-24"
        accelerator_count = 2
    else:
        raise ValueError(
            "Recommended machine settings not found for accelerator type: %s"
            % accelerator_type
        )
elif "27b" in base_model_id:
    if accelerator_type == "NVIDIA_L4":
        # Sets 4 L4 (24G) to deploy Gemma 2 27B models.
        machine_type = "g2-standard-48"
        accelerator_count = 4
    else:
        raise ValueError(
            "Recommended machine settings not found for accelerator type: %s"
            % accelerator_type
        )
else:
    raise ValueError(
        "Recommended machine settings not found for model: %s" % base_model_id
    )

common_util.check_quota(
    project_id=PROJECT_ID,
    region=REGION,
    accelerator_type=accelerator_type,
    accelerator_count=accelerator_count,
    is_for_training=False,
)

Set use_dedicated_endpoint to True if the endpoint is [dedicated endpoint](https://cloud.google.com/vertex-ai/docs/general/deployment#create-dedicated-endpoint) enabled.

In [ ]:
use_dedicated_endpoint = False  # @param {type:"boolean"}

Accelerator parameters

In [ ]:
gpu_memory_utilization = 0.85
max_model_len = 4096  # Maximum context length.

Deployment function

In [ ]:
def deploy_model_vllm(
    model_name: str,
    model_id: str,
    service_account: str,
    base_model_id: str = None,
    machine_type: str = "g2-standard-8",
    accelerator_type: str = "NVIDIA_L4",
    accelerator_count: int = 1,
    gpu_memory_utilization: float = 0.9,
    max_model_len: int = 4096,
    dtype: str = "auto",
    enable_trust_remote_code: bool = False,
    enforce_eager: bool = False,
    enable_lora: bool = False,
    max_loras: int = 1,
    max_cpu_loras: int = 8,
    use_dedicated_endpoint: bool = False,
    max_num_seqs: int = 256,
    model_type: str = None,
) -> Tuple[aiplatform.Model, aiplatform.Endpoint]:
    """Deploys trained models with vLLM into Vertex AI."""
    endpoint = aiplatform.Endpoint.create(
        display_name=f"{model_name}-endpoint",
        dedicated_endpoint_enabled=use_dedicated_endpoint,
    )

    if not base_model_id:
        base_model_id = model_id

    # See https://docs.vllm.ai/en/latest/models/engine_args.html for a list of possible arguments with descriptions.
    vllm_args = [
        "python",
        "-m",
        "vllm.entrypoints.api_server",
        "--host=0.0.0.0",
        "--port=8080",
        f"--model={model_id}",
        f"--tensor-parallel-size={accelerator_count}",
        "--swap-space=16",
        f"--gpu-memory-utilization={gpu_memory_utilization}",
        f"--max-model-len={max_model_len}",
        f"--dtype={dtype}",
        f"--max-loras={max_loras}",
        f"--max-cpu-loras={max_cpu_loras}",
        f"--max-num-seqs={max_num_seqs}",
        "--disable-log-stats",
    ]

    if enable_trust_remote_code:
        vllm_args.append("--trust-remote-code")

    if enforce_eager:
        vllm_args.append("--enforce-eager")

    if enable_lora:
        vllm_args.append("--enable-lora")

    if model_type:
        vllm_args.append(f"--model-type={model_type}")

    env_vars = {
        "MODEL_ID": base_model_id,
        "DEPLOY_SOURCE": "notebook",
    }

    # HF_TOKEN is not a compulsory field and may not be defined.
    try:
        if HF_TOKEN:
            env_vars["HF_TOKEN"] = HF_TOKEN
    except NameError:
        pass

    model = aiplatform.Model.upload(
        display_name=model_name,
        serving_container_image_uri=VLLM_DOCKER_URI,
        serving_container_args=vllm_args,
        serving_container_ports=[8080],
        serving_container_predict_route="/generate",
        serving_container_health_route="/ping",
        serving_container_environment_variables=env_vars,
        serving_container_shared_memory_size_mb=(16 * 1024),  # 16 GB
        serving_container_deployment_timeout=7200,
    )
    print(
        f"Deploying {model_name} on {machine_type} with {accelerator_count} {accelerator_type} GPU(s)."
    )
    model.deploy(
        endpoint=endpoint,
        machine_type=machine_type,
        accelerator_type=accelerator_type,
        accelerator_count=accelerator_count,
        deploy_request_timeout=1800,
        service_account=service_account,
    )
    print("endpoint_name:", endpoint.name)

    return model, endpoint

Deploy

In [ ]:
models["vllm_gpu"], endpoints["vllm_gpu"] = deploy_model_vllm(
    model_name=common_util.get_job_name_with_datetime(prefix="gemma2-vllm-serve"),
    model_id=merged_model_output_dir,
    service_account=SERVICE_ACCOUNT,
    machine_type=machine_type,
    accelerator_type=accelerator_type,
    accelerator_count=accelerator_count,
    gpu_memory_utilization=gpu_memory_utilization,
    max_model_len=max_model_len,
    use_dedicated_endpoint=use_dedicated_endpoint,
)

### Predict

Once deployment succeeds, you can send requests to the endpoint with text prompts. Sampling parameters supported by vLLM can be found [here](https://docs.vllm.ai/en/latest/dev/sampling_params.html).

Additionally, you can moderate the generated text with Vertex AI. See [Moderate text documentation](https://cloud.google.com/natural-language/docs/moderating-text) for more details.

**[Optional]** Loads an existing endpoint instance using the endpoint name:
- Using `endpoint_name = endpoint.name` allows us to get the endpoint name of the endpoint `endpoint` created in the cells above.
- Alternatively, you can set `endpoint_name = "1234567890123456789"` to load an existing endpoint with the ID 1234567890123456789.
You may uncomment the code below to load an existing endpoint.

In [ ]:
# endpoint_name = ""  # @param {type:"string"}
# aip_endpoint_name = (
#     f"projects/{PROJECT_ID}/locations/{REGION}/endpoints/{endpoint_name}"
# )
# endpoint = aiplatform.Endpoint(aip_endpoint_name)

Here we use an Example:

```
Prompt: What is a car?
Output:  A car, or a motor car, is a road-connected human-transportation system used to move people or goods from one place to another. The term also encompasses a wide range of vehicles, including motorboats, trains, and aircrafts. Cars typically have four wheels, a cabin for passengers, and an engine or motor. They have been around since the early 19th century and are now one of the most popular forms of transportation, used for daily commuting, shopping, and other purposes.
```

Prompt

In [ ]:
prompt = "What is a car?"  # @param {type: "string"}

Generation parameters

In [ ]:
max_tokens = 50  # @param {type:"integer"}
temperature = 1.0  # @param {type:"number"}
top_p = 1.0  # @param {type:"number"}
top_k = 1  # @param {type:"integer"}
raw_response = False  # @param {type:"boolean"}

Note: If you encounter the issue like `ServiceUnavailable: 503 Took too long to respond when processing`, you can reduce the maximum number of output tokens, by lowering `max_tokens`.

In [ ]:
# Overrides parameters for inferences.
instances = [
    {
        "prompt": prompt,
        "max_tokens": max_tokens,
        "temperature": temperature,
        "top_p": top_p,
        "top_k": top_k,
        "raw_response": raw_response,
    },
]

Predict

In [ ]:
response = endpoints["vllm_gpu"].predict(
    instances=instances, use_dedicated_endpoint=use_dedicated_endpoint
)

for prediction in response.predictions:
    print(prediction)

## Clean up resources

Delete the experiment models and endpoints to recycle the resources and avoid unnecessary continuous charges that may incur.

In [ ]:
# Delete the train job.
train_job.delete()

# Undeploy model and delete endpoint.
for endpoint in endpoints.values():
    endpoint.delete(force=True)

# Delete models.
for model in models.values():
    model.delete()

If you want to delete the bucket, change the variable to True.

In [ ]:
delete_bucket = False  # @param {type:"boolean"}
if delete_bucket:
    ! gsutil -m rm -r $BUCKET_NAME